In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from joblib import load
from sklearn.model_selection import train_test_split
from utils.dscarnet import dual_dscarnet

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
)

In [3]:
data = pd.read_csv('dataset/data.csv', low_memory = False)
data['label'] = data['label'].replace({r'\b(PEPH|NaPE)\b': 'PE', r'\b(peph)\b': 'pe'}, regex = True)
data['label'] = data['label'].str.strip()
mask = (data['label'] == '0') | (data['label'].str.contains('/', na=False))
data_nontarget_index, data_target_index = data.index[mask], data.index[~mask]

X1 = load('dataset/sherloc_x1.data')
X2 = load('dataset/sherloc_x2.data')

classes = ['OL', 'PE', 'PX', 'SU', 'SI', 'CA']
class2idx = {c: i for i, c in enumerate(classes)}

def multilabel_to_vector(label_str):
    vec = np.zeros(len(classes))
    s = str(label_str).strip()
    if s == '0' or s == ' ':
        return vec
    for lbl in s.split("/"):
        lbl = lbl.strip().upper()
        idx = class2idx.get(lbl)
        if idx is not None:
            vec[idx] = 1
    return vec

Y = np.stack([multilabel_to_vector(lbl) for lbl in data['label']])

In [4]:
df_target_idx = pd.RangeIndex(0, 4501).intersection(data_target_index)

train_idx, temp_idx = train_test_split(
    df_target_idx,
    test_size = 0.2,
    random_state = 42,
    stratify = data.loc[df_target_idx, 'label']
)
valid_idx, test_idx = train_test_split(
    temp_idx,
    test_size = 0.5,
    random_state = 42,
    stratify = data.loc[temp_idx, 'label']
)
        
X1_train, X2_train, Y_train = X1[train_idx], X2[train_idx], Y[train_idx]
X1_valid, X2_valid, Y_valid = X1[valid_idx], X2[valid_idx], Y[valid_idx]
X1_test,  X2_test,  Y_test  = X1[test_idx],  X2[test_idx],  Y[test_idx]

trainX, validX, testX = (X1_train, X2_train), (X1_valid, X2_valid), (X1_test, X2_test)
trainY, validY, testY = Y_train, Y_valid, Y_test

In [5]:
def make_weighted_ce(class_weights, label_smoothing = 0.05):
    w = tf.constant(class_weights, dtype = tf.float32)
    ce = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing = label_smoothing,
        reduction = tf.keras.losses.Reduction.NONE
    )
    def loss_fn(y_true, y_pred):
        per_sample = ce(y_true, y_pred)             
        sample_w = tf.reduce_sum(w * y_true, axis = -1)  
        return tf.reduce_mean(per_sample * sample_w)
    return loss_fn

In [6]:
positive_counts = np.maximum((trainY > 0).sum(axis = 0), 1)      
orig_weights = trainY.shape[0] / (len(positive_counts) * positive_counts)
median_w = np.median(orig_weights)
class_weights = np.minimum(orig_weights, median_w * 10)

model = dual_dscarnet(X1.shape[1:], X2.shape[1:], n_outputs = len(classes), last_avf = 'softmax')
opt = tf.keras.optimizers.Adam(learning_rate = 1e-4, clipnorm = 1.0)
loss = make_weighted_ce(class_weights, label_smoothing = 0.05)
model.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 20, min_lr = 1e-6, verbose = 1),
    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 50, restore_best_weights = True, verbose = 1)
]

2025-12-01 17:31:53.002804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 17:31:53.591434: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:cd:00.0, compute capability: 8.0


In [7]:
history = model.fit(
    trainX, trainY,
    validation_data = (validX, validY),
    batch_size = 32,
    epochs = 10,
    shuffle = True,
    verbose = 1,
    callbacks = callbacks
)

Epoch 1/10


2025-12-01 17:31:56.744121: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2025-12-01 17:31:58.220750: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


59/59 [==============================] - 7s 43ms/step - loss: 1.6993 - accuracy: 0.3684 - val_loss: 1.7455 - val_accuracy: 0.1202 - lr: 1.0000e-04
Epoch 2/10
59/59 [==============================] - 2s 31ms/step - loss: 1.4642 - accuracy: 0.5446 - val_loss: 1.4000 - val_accuracy: 0.5794 - lr: 1.0000e-04
Epoch 3/10
59/59 [==============================] - 2s 32ms/step - loss: 1.2733 - accuracy: 0.6074 - val_loss: 1.2278 - val_accuracy: 0.6867 - lr: 1.0000e-04
Epoch 4/10
59/59 [==============================] - 2s 32ms/step - loss: 1.1175 - accuracy: 0.7143 - val_loss: 1.1002 - val_accuracy: 0.7725 - lr: 1.0000e-04
Epoch 5/10
59/59 [==============================] - 2s 32ms/step - loss: 0.9994 - accuracy: 0.7830 - val_loss: 0.9853 - val_accuracy: 0.8069 - lr: 1.0000e-04
Epoch 6/10
59/59 [==============================] - 2s 33ms/step - loss: 0.9135 - accuracy: 0.8077 - val_loss: 0.9310 - val_accuracy: 0.8283 - lr: 1.0000e-04
Epoch 7/10
59/59 [==============================] - 2s 26ms/ste